# Protein Gym Exploration

From the huggingface, markdown file. Found [here](https://huggingface.co/datasets/OATML-Markslab/ProteinGym/blob/main/reference_files_description.md)
## ProteinGym reference files

In the reference files, we provide detailed information about all DMS assays included in ProteinGym. There are two reference files: one for the substitution benchmark and one for the indel benchmark.

The meaning of each column in the ProteinGym reference files is provided below:
- DMS_id (str): Uniquely identifies each DMS assay in ProteinGym. It is obtained as the concatenation of the UniProt ID of the mutated protein, the first author name and the year of publication. If there are several datasets with the same characteristics, another defining attribute of the assay is added to preserve unicity.
- DMS_filename (str): Name of the processed DMS file.
- target_seq (str): Sequence of the target protein (reference sequence mutated in the assay).
- seq_len (int): Length of the target protein sequence.
- includes_multiple_mutants (bool): Indicates whether the DMS contains mutations that are multiple mutants. Substitution benchmark only.
- DMS_total_number_mutants (int): Number of rows of the DMS in ProteinGym.
- DMS_number_single_mutants (int): Number of single amino acid substitutions in the DMS. Substitution benchmark only.
- DMS_number_multiple_mutants (int): Number of multiple amino acid substitutions in the DMS. Substitution benchmark only.
- DMS_binarization_cutoff_ProteinGym (float): Cutoff used to divide fitness scores into binary labels.
- DMS_binarization_method (str): Method used to decide the binarization cutoff (manual or median).
- region_mutated (str): Region of the target protein that is mutated in the DMS.
- MSA_filename (str): Name of the MSA file generated based on the reference sequence mutated during the DMS experiment. Note that different reference sequences may be used in different DMS experiments for the same protein. For example, Giacomelli et al. (2018) and Kotler et al. (2018) used slightly different reference sequences in their respective DMS experiments for the P53 protein. We generated different MSAs accordingly.
- MSA_start (int): Locates the beginning of the first sequence in the MSA with respect to the target sequence. For example, if the MSA covers from position 10 to position 60 of the target sequence, then MSA_start is 10.
- MSA_end (int): Locates the end of the first sequence in the MSA with respect to the target sequence. For example, if the MSA covers from position 10 to position 60 of the target sequence, then MSA_end is 60.
- MSA_bitscore (float): Bitscore threshold used to generate the alignment divided by the length of the target protein.
- MSA_theta (float): Hamming distance cutoff for sequence re-weighting.
- MSA_num_seqs (int): Number of sequences in the Multiple Sequence Alignment (MSA) used in this work for this DMS.
- MSA_perc_cov (float): Percentage of positions of the MSA that had a coverage higher than 70% (less than 30% gaps).
- MSA_num_cov (int): Number of positions of the MSA that had a coverage higher than 70% (less than 30% gaps).
- MSA_N_eff (float): The effective number of sequences in the MSA defined as the sum of the different sequence weights.
- MSA_N_eff_L (float): Neff / num_cov.
- MSA_num_significant (int): Number of evolutionary couplings that are considered significant. Significance is defined by having more than 90% probability of belonging to the log-normal distribution in a Gaussian Mixture Model of normal and log-normal distributions.
- MSA_num_significant_L (float): MSA_num_significant / num_cov.
- raw_DMS_filename (str): Name of the raw DMS file.
- raw_DMS_phenotype_name (str): Name of the column in the raw DMS that we used as fitness score.
- raw_DMS_directionality (int): Sign of the correlation between the DMS_phenotype column values and protein fitness in the raw DMS files. In any given DMS, the directionality is 1 if higher values of the measurement are associated with higher fitness, and -1 otherwise. For simplicity, we adjusted directionality in the final ProteinGym benchmarks so that a higher value of DMS_score is always associated with higher fitness. Consequently, correlations between model scores and the final DMS_score values should always be positive (unless the predictions from the considered model are worse than random for that DMS).
- raw_DMS_mutant_column (str): Name of the column in the raw DMS that indicates which mutants were assayed.

## Code

In [1]:
# system dependencies
import os

# library dependencies
from datasets import load_dataset, list_datasets,  load_dataset_builder, get_dataset_split_names, get_dataset_config_names
from tqdm import tqdm
import numpy as np
import pandas as pd

# local dependencies

/gscratch/cheme/users/halanzi/mambaforge/envs/nomelt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# datasets_list = list_datasets()

/tmp/ipykernel_89433/3387996211.py:1: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  datasets_list = list_datasets()


In [11]:
# print(', '.join(dataset for dataset in datasets_list))

In [7]:
# let's see if we can download the dataset
dataset = load_dataset("OATML-Markslab/ProteinGym", split="train", cache_dir="../tmp/hf_cache/", data_dir="../data/gym/")

Repo card metadata block was not found. Setting CardData to empty.
Resolving data files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 882.38it/s]































































































Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.58it/s]
Generating train split: 265594 examples [00:02, 129138.44 examples/s]


DatasetGenerationError: An error occurred while generating the dataset

In [13]:
dataset = load_dataset("OATML-Markslab/ProteinGym", split="train")

Repo card metadata block was not found. Setting CardData to empty.
Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.37s/it]
Generating train split: 265594 examples [00:02, 123557.04 examples/s]


DatasetGenerationError: An error occurred while generating the dataset

Hmmm. It seems I am having an issue that I am not sure what the source is.

Let's inspect the dataset

In [15]:
ds_builder = load_dataset_builder("OATML-Markslab/ProteinGym")

Repo card metadata block was not found. Setting CardData to empty.
Resolving data files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 240533.56it/s]


In [16]:
ds_builder.info.description

''

In [17]:
ds_builder.info.features

In [20]:
get_dataset_split_names("OATML-Markslab/ProteinGym")

Repo card metadata block was not found. Setting CardData to empty.
Resolving data files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 805.32it/s]


['train']

In [22]:
configs = get_dataset_config_names("OATML-Markslab/ProteinGym")
print(configs)

Repo card metadata block was not found. Setting CardData to empty.
Resolving data files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 860.62it/s]


['default']


---

## Manual 

- I will just download it manually lol
- I will start w/ substitutions

In [2]:
from datasets import DownloadManager
import shutil

In [24]:
# # manually curate csv files (the following are just from the indel folder)
# # in principle, you can do this for substitutions as well
# # I also avoided human + yeast proteins
# csv_files = [
#     "A0A1J4YT16_9PROT_Davidi_2020.csv",  # Replace with actual file names if known
#     # "B1LPA6_ECOSM_Russ_2020.csv",
#     # "BLAT_ECOLX_Gonzalez_indels_2019.csv",
#     # "CAPSD_AAV2S_Sinai_indels_2021.csv",
# ]

In [12]:
csv_files = [
    "BLAT_ECOLX_Deng_2012.csv"
]

In [11]:
# Base URL for the ProteinGym_substitutions folder
base_url = "https://huggingface.co/datasets/OATML-Markslab/ProteinGym/raw/main/ProteinGym_substitutions/"

In [13]:
# Specify a directory to store the downloaded data
download_dir = "../data/gym/"
try:
    os.makedirs(download_dir, exist_ok=True)
    print("download directory has been created (or already exists!)")
except OSError as e:
    print(f'Error creating directory: {e}')


# Initialize the download manager
download_manager = DownloadManager(dataset_name="ProteinGym")

# Attempt to download the csv files again
downloaded_paths = {}

for file_name in csv_files:
    data_url = base_url + file_name
    try:
        downloaded_file_path = download_manager.download(data_url)
        downloaded_paths[file_name] = downloaded_file_path
        print(f"downloaded {data_url}")
        # After downloading
        shutil.move(downloaded_file_path, os.path.join(download_dir, file_name))
        print(f"downloaded file moved from the huggingface cache to {download_dir}")
    except Exception as e:
        downloaded_paths[file_name] = f"Error: {e}"

download directory has been created (or already exists!)


downloaded https://huggingface.co/datasets/OATML-Markslab/ProteinGym/raw/main/ProteinGym_substitutions/BLAT_ECOLX_Deng_2012.csv


### Code Draft (so far)

In [ ]:
# system dependencies
import os
import shutil

# library dependencies
from datasets import DownloadManager
from tqdm import tqdm
import numpy as np
import pandas as pd

# local dependencies

## constants
csv_files = [
    "BLAT_ECOLX_Deng_2012.csv"
] # initalize CSVs
# Base URL for the ProteinGym_substitutions folder
base_url = "https://huggingface.co/datasets/OATML-Markslab/ProteinGym/raw/main/ProteinGym_substitutions/"

# Specify a directory to store the downloaded data
download_dir = "../data/gym/"
try:
    os.makedirs(download_dir, exist_ok=True)
    print("download directory has been created (or already exists!)")
except OSError as e:
    print(f'Error creating directory: {e}')


# Initialize the download manager
download_manager = DownloadManager(dataset_name="ProteinGym")

# Attempt to download the csv files again
downloaded_paths = {}

for file_name in csv_files:
    data_url = base_url + file_name
    try:
        downloaded_file_path = download_manager.download(data_url)
        downloaded_paths[file_name] = downloaded_file_path
        print(f"downloaded {data_url}")
        # After downloading
        shutil.move(downloaded_file_path, os.path.join(download_dir, file_name))
        print(f"downloaded file moved from the huggingface cache to {download_dir}")
    except Exception as e:
        downloaded_paths[file_name] = f"Error: {e}"


#### Conclusion
- The draft above has been made to a script called `testing/download_protein_gym.py`
    - Not the most robust script of all time, but gets the job done! MVP! (Minimal viable product)

## Reading DMS data and analyzing it

- nearest analysis to what I am about to do can be found in `analysis/model_liklihood_for_known_mutants.ipynb`

In [1]:
# system dependencies
import io
import os
import sys
import re
from yaml import safe_load

# library dependencies
import duckdb as ddb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
import codecarbon

# local dependencies
from nomelt.model import NOMELTModel

/gscratch/cheme/users/halanzi/mambaforge/envs/nomelt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-16 16:32:43.917593: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-16 16:32:43.952595: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# seaborn settings
sns.set_context("talk")
sns.set_style("whitegrid")

### Reading the data
- I have a couple of choices. I can read via datasets from disk, duckdb, or Pandas. For now, I will work w/ Pandas for simplicity.

In [4]:
data = pd.read_csv("../data/gym/ESTA_BACSU_Nutschel_2020.csv")

In [5]:
data.head()

,mutant,DMS_score,DMS_score_bin
0,A32N,48.60,1
1,A32E,46.44,1
2,A32F,47.46,1
3,A32G,47.91,1
4,A32H,47.77,1


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2172 entries, 0 to 2171
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   mutant         2172 non-null   object 
 1   DMS_score      2172 non-null   float64
 2   DMS_score_bin  2172 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 51.0+ KB


Not big at all!

In [7]:
data.describe()

,DMS_score,DMS_score_bin
count,2172.000000,2172.000000
mean,46.072463,0.503223
std,2.448229,0.500105
min,40.020000,0.000000
25%,44.470000,0.000000
50%,46.340000,1.000000
75%,47.790000,1.000000
max,55.980000,1.000000


In [8]:
data["mutant"]

0        A32N
1        A32E
2        A32F
3        A32G
4        A32H
        ...  
2167    N205K
2168    N205L
2169    N205M
2170    N205A
2171    N205T
Name: mutant, Length: 2172, dtype: object

In [9]:
data["mutant"].iloc[0]

'A32N'

### Thinking zone (outdated. I was using Cheng's dataset)
- Doesn't tell me the full sequence mutation only what was mutated and at what position and to what.
    - In the above case, a histidine (a postively charged AA) was mutated to cysteine (the sulfide AA, special case!)
- On a closer inspection, if we notice what the CSV is called, we find it is called `BLAT_ECOLX`.
    - if we google this, we find from the following [website](https://www.genome.jp/dbget-bin/www_bget?sp:BLAT_ECOLX), this is a protein (Beta-lactamase TEM) which is found in _E.coli_.(checks out)
    - Given that this is from a model organism, it most likely has many proteoforms henceforth many reference proteomes. Thankfully, in the curation of the dataset, we limited these insitences to only one!
    - Now I just to find it lol
- Given that these are all single-point mutations of a single protein. In principle, I can make a mutation table for this, and what I am predicting is essentially DMS instead of Tm (?)

- Because I am lazy, I will assume the protein is the [P62593](https://www.uniprot.org/uniprotkb/P62593/entry). This was last updated in 1986!!!! So sequencing was in 86 and DMS analysis was 2015. Standing on top of gaints!
    - *Update* checking the reference file, I was right :)

In [10]:
wt = "MKFVKRRIIALVTILMLSVTSLFALQPSAKAAEHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGTNYNNGPVLSRFVQKVLDETGAKKVDIVAHSMGGANTLYYIKNLDGGNKVANVVTLGGANRLTTGKALPGTDPNQKILYTSIYSSADMIVMNYLSRLDGARNVQIHGVGHIGLLYSSQVNSLIKEGLNGGGQNTN"

In [11]:
len(wt)

212

In [12]:
wt[31]

'A'

Presumably the length of this protein is 286 AAs, which tracks with our mutation list 

In [13]:
mutations = data["mutant"].iloc[:]

In [14]:
mutations

0        A32N
1        A32E
2        A32F
3        A32G
4        A32H
        ...  
2167    N205K
2168    N205L
2169    N205M
2170    N205A
2171    N205T
Name: mutant, Length: 2172, dtype: object

In [15]:
def generate_mutated_sequences(wt_sequence: str, mutation_table_path: str) -> list:
    """
    Generate a list of mutated sequences from a mutation table, each with a single mutation applied to the wild-type.

    Parameters:
    - wt_sequence: The wild-type protein sequence.
    - mutation_table_path: Path to the CSV containing the mutation table.

    Returns:
    - List of mutated protein sequences.
    """
    # Load the CSV
    mutation_df = pd.read_csv(mutation_table_path)

    # Extract mutations
    mutations = mutation_df["mutant"]

    # List to store all mutated sequences
    all_mutated_sequences = []

    # Apply each mutation
    for mutation in mutations:
        # Create a fresh copy of wt for each mutation
        mutated_sequence = list(wt_sequence)
        
        match = re.match(r'([A-Z]+)(\d+)([A-Z]+)', mutation)
        if not match:
            continue # Skip any mutations that don't match the expected format
        original_aa, position, new_aa = match.groups() # Return a part of the string where there was a match
        position = int(position) - 1 # Convert to 0-based index

        # Check if the current amino acid matches the expected amino acid from the mutation table
        if mutated_sequence[position] != original_aa:
            print(f"Discrepancy at position {position+1}: Expected {original_aa}, Found {mutated_sequence[position]}, Mutation Table suggests changing to {new_aa}.")
            continue

        # Apply the mutation
        mutated_sequence[position] = new_aa

        # Append to the list of all mutated sequences
        all_mutated_sequences.append(''.join(mutated_sequence))

    return all_mutated_sequences

In [16]:
mutated_sequences_list = generate_mutated_sequences(wt, "../data/gym/ESTA_BACSU_Nutschel_2020.csv")
mutated_sequences_list[:5]

['MKFVKRRIIALVTILMLSVTSLFALQPSAKANEHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGTNYNNGPVLSRFVQKVLDETGAKKVDIVAHSMGGANTLYYIKNLDGGNKVANVVTLGGANRLTTGKALPGTDPNQKILYTSIYSSADMIVMNYLSRLDGARNVQIHGVGHIGLLYSSQVNSLIKEGLNGGGQNTN',
 'MKFVKRRIIALVTILMLSVTSLFALQPSAKAEEHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGTNYNNGPVLSRFVQKVLDETGAKKVDIVAHSMGGANTLYYIKNLDGGNKVANVVTLGGANRLTTGKALPGTDPNQKILYTSIYSSADMIVMNYLSRLDGARNVQIHGVGHIGLLYSSQVNSLIKEGLNGGGQNTN',
 'MKFVKRRIIALVTILMLSVTSLFALQPSAKAFEHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGTNYNNGPVLSRFVQKVLDETGAKKVDIVAHSMGGANTLYYIKNLDGGNKVANVVTLGGANRLTTGKALPGTDPNQKILYTSIYSSADMIVMNYLSRLDGARNVQIHGVGHIGLLYSSQVNSLIKEGLNGGGQNTN',
 'MKFVKRRIIALVTILMLSVTSLFALQPSAKAGEHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGTNYNNGPVLSRFVQKVLDETGAKKVDIVAHSMGGANTLYYIKNLDGGNKVANVVTLGGANRLTTGKALPGTDPNQKILYTSIYSSADMIVMNYLSRLDGARNVQIHGVGHIGLLYSSQVNSLIKEGLNGGGQNTN',
 'MKFVKRRIIALVTILMLSVTSLFALQPSAKAHEHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGTNYNNGPVLSRFVQKVLDETGAKKVDIVAHSMGGANTLYYIKNLDGGNKVANV

In [17]:
for _, seq in enumerate(mutated_sequences_list):
    print(seq)

...


In [22]:
dir(mutated_sequences_list)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [23]:
type(mutated_sequences_list)

list

In [33]:
test = [seq for _,seq in enumerate(mutated_sequences_list)]

In [38]:
type(test)

list

In [40]:
len(test)

2172

In [39]:
len(mutated_sequences_list)

2172

According to the DMS reference file for substitution benchmarking, this is correct!

In [19]:
data

,mutant,DMS_score,DMS_score_bin
0,H24C,-3.924478,0
1,H24Y,-2.170022,1
2,H24W,-4.345218,0
3,H24V,-3.011503,0
4,H24T,-1.565248,1
...,...,...,...
4991,W286G,-4.963070,0
4992,W286F,-4.542329,0
4993,W286E,-4.542329,0
4994,W286C,-4.542329,0


make list to a pandas series

In [50]:
test_seq = pd.Series(mutated_sequences_list).values

In [51]:
test_seq

array(['MKFVKRRIIALVTILMLSVTSLFALQPSAKANEHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGTNYNNGPVLSRFVQKVLDETGAKKVDIVAHSMGGANTLYYIKNLDGGNKVANVVTLGGANRLTTGKALPGTDPNQKILYTSIYSSADMIVMNYLSRLDGARNVQIHGVGHIGLLYSSQVNSLIKEGLNGGGQNTN',
       'MKFVKRRIIALVTILMLSVTSLFALQPSAKAEEHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGTNYNNGPVLSRFVQKVLDETGAKKVDIVAHSMGGANTLYYIKNLDGGNKVANVVTLGGANRLTTGKALPGTDPNQKILYTSIYSSADMIVMNYLSRLDGARNVQIHGVGHIGLLYSSQVNSLIKEGLNGGGQNTN',
       'MKFVKRRIIALVTILMLSVTSLFALQPSAKAFEHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGTNYNNGPVLSRFVQKVLDETGAKKVDIVAHSMGGANTLYYIKNLDGGNKVANVVTLGGANRLTTGKALPGTDPNQKILYTSIYSSADMIVMNYLSRLDGARNVQIHGVGHIGLLYSSQVNSLIKEGLNGGGQNTN',
       ...,
       'MKFVKRRIIALVTILMLSVTSLFALQPSAKAAEHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGTNYNNGPVLSRFVQKVLDETGAKKVDIVAHSMGGANTLYYIKNLDGGNKVANVVTLGGANRLTTGKALPGTDPNQKILYTSIYSSADMIVMNYLSRLDGARNVQIHGVGHIGLLYSSQVNSLIKEGLMGGGQNTN',
       'MKFVKRRIIALVTILMLSVTSLFALQPSAKAAEHNPVVMVHGIGGASFNFAGIKSYLVSQGWSRDKLYAVDFWDKTGTNYNNGPVLSR

### Model scoring

In [20]:
with open('../params.yaml', 'r') as f:
    params = safe_load(f)

In [21]:
hyperparams = params['model']['model_hyperparams']

In [22]:
hyperparams

{'dropout_rate': 0.1, 'relative_attention_max_distance': 250}

In [23]:
model = NOMELTModel('../data/nomelt-model/model', **hyperparams)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/gscratch/cheme/users/halanzi/mambaforge/envs/nomelt/lib/python3.9/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Testing Evan's function as is. This wasn't designed with DMS in mind, but let's see what happens!

In [ ]:
wt_score, variant_scores = model.score_variants(wt, mutated_sequences_list)

Tokenizing and preparing model inputs: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4997/4997 [00:06<00:00, 767.86 examples/s]
Generating train split: 0 examples [00:00, ? examples/s]/mmfs1/gscratch/cheme/users/halanzi/git_repos/nomelt/nomelt/model.py:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  yield {'loss': torch.tensor(loss).reshape(1,-1), 'logits': logit}
Generating train split: 1160 examples [1:12:08,  3.73s/ examples]


KeyboardInterrupt: 

In [24]:
x_data = data["DMS_score"]

,mutant,DMS_score,DMS_score_bin
0,H24C,-3.924478,0
1,H24Y,-2.170022,1
2,H24W,-4.345218,0
3,H24V,-3.011503,0
4,H24T,-1.565248,1
...,...,...,...
4991,W286G,-4.963070,0
4992,W286F,-4.542329,0
4993,W286E,-4.542329,0
4994,W286C,-4.542329,0


In [25]:
y_data = [0.0] + variant_scores

NameError: name 'variant_scores' is not defined

In [ ]:
ax, fig = plt.subplots(figsize(6,6))
sns.regplot(x=x_data, y=y_data)
ax.set_title('Figure 1: model vs DMS')
ax.set_xlabel('DMS score (exp)')
ax.set_ylabel('Model score')
plt.show()